In [0]:

# from applications.db_check_v1.common.db_check_utils import query, pg_settings, etl_skip

from conf import settings


def pg_settings(schema):
    template = "PG_" + schema.upper() + "_{property}"
    host, port = getattr(settings, template.format(property='HOSTS'))[0]
    return (
        "dbname='{db}' user='{user}' password='{password}' "
        "host='{host}' port='{port}'".format(
            db=getattr(settings, template.format(property='NAME')),
            user=getattr(settings, template.format(property='ACCESS_ID')),
            host=host,
            password=getattr(settings, template.format(property='SECRET_KEY')),
            port=port
        )
    )

_pg_dsn = pg_settings('aa_android')

print _pg_dsn
sql_gp = "SELECT * FROM app limit 1"
gp_res = query(_pg_dsn, sql_gp)



In [0]:
%%sh
PGPASSWORD='wZw8cfBuuklIskVG' psql -h 10.2.6.141  -U citus_bdp_prod_app_int_qa -d aa_store_db -p 5432 << EOF 
set search_path=store;
-- select * from store_est_t_w_fact_v1 where date between '2020-04-08' and '2020-04-09' and app_id In (835599320) and country_code='WW' and device_code='ios-phone' ;

--select * from store_est_category_fact_v1 where date between '2019-08-01' and '2019-08-01' and app_id In (1512799121) and country_code='US' order by date desc limit 5 ;
-- select * from store_est_t_m_fact_v2 where date between '2020-03-31' and '2020-06-20' and app_id In (835599320) and country_code='US' and device_code='ios-phone'  order by date desc limit 5 ;
--select count(*) from store_est_t_w_fact_v2 where app_id In (835599320) and country_code='US' and device_code='ios-phone'  limit 10 ;
--select count(*) from store_est_t_w_fact_v1 where app_id In (835599320) and country_code='US' and device_code='ios-phone'  limit 10 ;
select * from store_est_fact_v2 where app_id=1353334793 limit 3;
EOF

In [0]:
%%sh
# aws s3 ls s3://b2c-prod-dca-store-estimates/store_estv2/APP_ESTIMATES_FINAL/
aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.product-category-download-revenue.v1/fact/granularity_code=daily/date=2021-01-01/data_stage=final/market_code=apple-store/device_code=ios-phone/


In [0]:


#spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.product-category-download-revenue.v1/fact/granularity_code=daily/date=2021-01-01/data_stage=final/market_code=apple-store/device_code=ios-phone/").createOrReplaceTempView("temp")
# spark.sql("select* from temp limit 1").show()
spark.sql("select product_key, est_free_app_download from temp where _identifier='120210112054630395' and category_key=100000 and country_code='JP' and product_key in (1483058899, 1435783608,443904275,546505307,405489166, 544007664)").show()



In [0]:

# print spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-publisher-dna-log.v1/fact").where("granularity='daily' and date='2020-08-10' and device_code != 'ios-all'").count()
COUNTRY_CODE_MAPPING_BY_MARKET_CODE = {
    'google-play': {1: 'AU', 2: 'CA', 3: 'CN', 4: 'DE', 5: 'ES', 6: 'FR', 7: 'GB', 8: 'IT', 9: 'JP', 10: 'US',
                    11: 'BE', 12: 'CH', 13: 'CL', 14: 'ZA', 15: 'VN', 16: 'HK', 17: 'AR', 18: 'BR', 19: 'IN',
                    20: 'FI', 21: 'ID', 22: 'RU', 23: 'NL', 24: 'MY', 25: 'TR', 26: 'MX', 27: 'KR', 28: 'PL',
                    29: 'TH', 30: 'TW', 31: 'PH', 32: 'SG', 33: 'EG', 34: 'SE', 35: 'AT', 36: 'CZ', 37: 'HU',
                    38: 'DK', 39: 'IE', 40: 'IL', 41: 'NZ', 42: 'NO', 43: 'PT', 44: 'RO', 45: 'SK', 46: 'GR',
                    47: 'BG', 48: 'UA', 49: 'AE', 50: 'KW', 51: 'SA', 52: 'CO', 53: 'KZ', 54: 'PK', 55: 'IQ',
                    56: 'PE', 57: 'MA', 58: 'BY', 59: 'DZ', 60: 'VE', 61: 'AZ', 62: 'EC', 63: 'JO', 64: 'CR',
                    65: 'LB', 66: 'BD', 67: 'GT', 68: 'RS', 69: 'DO', 70: 'IR', 71: 'OM', 72: 'BO', 73: 'QA',
                    74: 'NG', 75: 'SV', 76: 'KH', 77: 'PA', 78: 'LT', 79: 'TN', 80: 'HR', 81: 'JM', 82: 'LK',
                    83: 'HN', 84: 'PR', 85: 'UY', 86: 'LV', 87: 'BA', 88: 'KG', 89: 'PY', 90: 'MD', 91: 'NP',
                    92: 'TZ', 93: 'BH', 94: 'GH', 95: 'KE', 96: 'SI', 97: 'AM', 98: 'UZ', 99: 'TT', 100: 'MK',
                    101: 'YE', 102: 'MO', 103: 'LU', 1000: 'WW'},
    'apple-store': {143441: 'US', 143442: 'FR', 143443: 'DE', 143444: 'GB', 143445: 'AT', 143446: 'BE', 143447: 'FI',
                    143448: 'GR', 143449: 'IE', 143450: 'IT', 143451: 'LU', 143452: 'NL', 143453: 'PT', 143454: 'ES',
                    143455: 'CA', 143456: 'SE', 143457: 'NO', 143458: 'DK', 143459: 'CH', 143460: 'AU', 143461: 'NZ',
                    143462: 'JP', 143463: 'HK', 143464: 'SG', 143465: 'CN', 143466: 'KR', 143467: 'IN', 143468: 'MX',
                    143469: 'RU', 143470: 'TW', 143471: 'VN', 143472: 'ZA', 143473: 'MY', 143474: 'PH', 143475: 'TH',
                    143476: 'ID', 143477: 'PK', 143478: 'PL', 143479: 'SA', 143480: 'TR', 143481: 'AE', 143482: 'HU',
                    143483: 'CL', 143484: 'NP', 143485: 'PA', 143486: 'LK', 143487: 'RO', 143489: 'CZ', 143491: 'IL',
                    143492: 'UA', 143493: 'KW', 143494: 'HR', 143495: 'CR', 143496: 'SK', 143497: 'LB', 143498: 'QA',
                    143499: 'SI', 143501: 'CO', 143502: 'VE', 143503: 'BR', 143504: 'GT', 143505: 'AR', 143506: 'SV',
                    143507: 'PE', 143508: 'DO', 143509: 'EC', 143510: 'HN', 143511: 'JM', 143512: 'NI', 143513: 'PY',
                    143514: 'UY', 143515: 'MO', 143516: 'EG', 143517: 'KZ', 143518: 'EE', 143519: 'LV', 143520: 'LT',
                    143521: 'MT', 143523: 'MD', 143524: 'AM', 143525: 'BW', 143526: 'BG', 143528: 'JO', 143529: 'KE',
                    143530: 'MK', 143531: 'MG', 143532: 'ML', 143533: 'MU', 143534: 'NE', 143535: 'SN', 143536: 'TN',
                    143537: 'UG', 143538: 'AI', 143539: 'BS', 143540: 'AG', 143541: 'BB', 143542: 'BM', 143543: 'VG',
                    143544: 'KY', 143545: 'DM', 143546: 'GD', 143547: 'MS', 143548: 'KN', 143549: 'LC', 143550: 'VC',
                    143551: 'TT', 143552: 'TC', 143553: 'GY', 143554: 'SR', 143555: 'BZ', 143556: 'BO', 143557: 'CY',
                    143558: 'IS', 143559: 'BH', 143560: 'BN', 143561: 'NG', 143562: 'OM', 143563: 'DZ', 143564: 'AO',
                    143565: 'BY', 143566: 'UZ', 143568: 'AZ', 143571: 'YE', 143572: 'TZ', 143573: 'GH', 143575: 'AL',
                    143576: 'BJ', 143577: 'BT', 143578: 'BF', 143579: 'KH', 143580: 'CV', 143581: 'TD', 143582: 'CG',
                    143583: 'FJ', 143584: 'GM', 143585: 'GW', 143586: 'KG', 143587: 'LA', 143588: 'LR', 143589: 'MW',
                    143590: 'MR', 143591: 'FM', 143592: 'MN', 143593: 'MZ', 143594: 'NA', 143595: 'PW', 143597: 'PG',
                    143598: 'ST', 143599: 'SC', 143600: 'SL', 143601: 'SB', 143602: 'SZ', 143603: 'TJ', 143604: 'TM',
                    143605: 'ZW', 0: 'WW',
                    # New Country added @ 2020-06-11
                    143610: 'AF', 143612: 'BA', 143613: 'CD', 143527: 'CI', 143574: 'CM', 143614: 'GA', 143622: 'ZM',
                    143615: 'GE', 143617: 'IQ', 143567: 'LY', 143620: 'MA', 143619: 'ME', 143570: 'MM', 143488: 'MV',
                    143606: 'NR', 143500: 'RS', 143621: 'RW', 143608: 'TO', 143609: 'VU', 143624: 'XK'},
    'amazon-store': {
        'android-all': {
            'UK': 'GB',
        }
    }
}

ios_store_ids = [str(k) for k, v in COUNTRY_CODE_MAPPING_BY_MARKET_CODE["apple-store"].items() ]
gp_store_ids = [str(k) for k, v in COUNTRY_CODE_MAPPING_BY_MARKET_CODE["google-play"].items() ]

spark.read.option("basePath", "s3://b2c-prod-dca-store-estimates/store_estv2/PUB_ESTIMATES_PREVIEW/version=2.0.0/range_type=DAY/").parquet("s3://b2c-prod-dca-store-estimates/store_estv2/PUB_ESTIMATES_PREVIEW/version=2.0.0/range_type=DAY/date=2020-08-10").createOrReplaceTempView("temp")
spark.sql("select feed, sum(est) from temp where (platform_id = 0 and store_id in ('{}') and feed in (0,1,2) ) group by feed".format("','".join(gp_store_ids))).show()
spark.sql("select feed, sum(est) from temp where (platform_id = 1 and store_id in ('{}') and feed in (0,1,2,100,101,102) ) group by feed ".format("','".join(ios_store_ids))).show()



In [0]:
%%sh
PGPASSWORD='wZw8cfBuuklIskVG' psql -h 10.2.6.141  -U citus_bdp_prod_app_int_qa -d aa_store_db -p 5432 << EOF 
set search_path=store;

-- select  sum(est_free_app_download), sum(est_paid_app_download), sum(est_revenue) from store_est_publisher_fact_v2 where date='2020-08-10' limit 1;
-- select sum(est_free_app_download), sum(est_paid_app_download), sum(est_revenue) from store_est_publisher_t_w_fact_v2 where date = '2020-08-08' limit 1;

select * from pg_indexes where tablename='store_est_publisher_t_w_fact_v2';

EOF


In [0]:
%%sh

# aws s3 ls s3://aardvark-prod-pdx-mdm-to-int/basic_kpi/version=1.0.0/range_type=DAY/date=2020-08-01/platform=2/

aws s3 ls s3://aardvark-prod-pdx-mdm-to-int/basic_kpi/version=1.0.0/range_type=WEEK/date=2020-08-01/


In [0]:
%%sh
PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-usage-plproxy-internal-1640809782.us-east-1.elb.amazonaws.com -U app_bdp_usage_qa -d cohort -p 7432 << EOF 


select sum(cnt) from plproxy.execute_select(\$proxy\$select count(1) as cnt from
                        mw.domain_basic_d"
                        "where date = '2020-07-18' and device_code='up' and country_code='us' limit 1\$proxy\$)t(cnt bigint);
     

EOF





In [0]:
%%sh
PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-4-329644124.us-east-1.elb.amazonaws.com -U app_bdp_usage_qa -d dailyest -p 7432 << EOF 
            select distinct date, genre_id
            from plproxy.execute_select_nestloop(\$proxy\$
            select distinct date, genre_id
            from aa.genre_store_daily_estimate
            where date = '2020-01-01'
            \$proxy\$) tbl (date DATE, genre_id BIGINT);

EOF

#  select date, device_code from plproxy.execute_select(\$proxy\$select date, device_code
#                   from mw.deduplicated_w where device_code='ap' and date in ('2020-07-25')  limit 5 \$proxy\$)t (
# date date, device_code char )  limit 3;


In [0]:
%%sh
# aws s3 ls s3://b2c-prod-data-pipeline-unified-store-free/unified/app-tech.store.app-rank.v2/fact/date=2020-03-30/
aws s3 ls s3://b2c-prod-data-pipeline-unified-usage/unified/usage.genre-cross.v1/fact/granularity=monthly/date=2020-02-29/


In [0]:
%%sh
PGPASSWORD='TMV!PYT02X*w' psql -h 54.210.244.2 -p 5433 -U app_tomcat -d aa << EOF
-- \d app;
-- SELECT id, name, company FROM app LIMIT 1;
SELECT * FROM aa_app  LIMIT 1;
EOF

In [0]:
%%sh
PGPASSWORD='IEdwiF2hmqzdKK43' psql -h b2b-prod-uds-storage-meta-db-new.crlexxwtzodp.us-east-1.rds.amazonaws.com -p 5432 -Uapp_meta_qa -d metadb << EOF
begin;
-- select * from meta where table_name like '%store_est_t_%' and is_active=True limit 7;
--select * from meta where table_name = 'advancedreview_rev_v_dimension' and is_active=True ;
select table_name, app_stats -> 'latest_available_date' as latest_available_date, version, vendor from meta where table_name like '%store_est_t_%' and is_active=True limit 7;

commit;
EOF


In [0]:
%%sh
PGPASSWORD='90H1O7a9c2PQj7SC' psql -h b2b-prod-datapipeline-meta-new.crlexxwtzodp.us-east-1.rds.amazonaws.com -p 5432 -Uaa_datapipeline -d pipeline_meta << EOF
-- \d+ ;
-- select * from pipeline_meta_event where namespace='aa.store.app-est-qa-pre-agg.v1' limit 1;
select * from pipeline_meta_event where namespace='aa.qa.app-est-qa.v1' order by identifier desc limit 3;

EOF

In [0]:
%%sh
PGPASSWORD='2mHdFW6%#REu' psql -h 10.2.25.198  -U app_bdp_usage_qa -d dna -p 6432 << EOF 
set search_path=public;
select * from public.in_app_dna_info_mapping limit 3;

EOF




In [0]:
%%sh


PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-usage-plproxy-internal-1640809782.us-east-1.elb.amazonaws.com -U app_bdp_usage_qa -d cohort -p 7432 << EOF 
select date, device_code from plproxy.execute_select(\$proxy\$select date, device_code
                    from mw.deduplicated_w where device_code='ap' and date in ('2020-07-25')  limit 5 \$proxy\$)t (
                    date date, device_code char )  limit 3;

EOF
#  select device_code from plproxy.execute_select(\$proxy\$select device_code 
#                     from mw.category where device_code='up' limit 5 \$proxy\$)t (
#                     device_code char )  limit 3;




In [0]:
%%sh
PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-2-31838298.us-east-1.elb.amazonaws.com -U app_bdp_usage_qa -d cohort -p 7432 << EOF 
select * from plproxy.execute_select(\$proxy\$select country_code, count(distinct(city_id)) as cnt
              from mu.city_app_w where date='2019-08-17' and device_code = 'ap'
              group by country_code \$proxy\$) t (
               country_code char, cnt BIGINT) group by country_code, cnt order by country_code limit 3;
--select * from plproxy.execute_select(\$proxy\$select app_id, city_id, est_average_active_users 
--              from mu.city_app_m 
--                \$proxy\$) t (
--             app_id bigint, city_id bigint, est_average_active_users real) limit 3;

--select 3.5 :: BIGINT;
--select 2.5 :: BIGINT;
--select 1.5 :: BIGINT;
--select 1.4 :: BIGINT;

EOF



In [0]:
%%sh


PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-2-31838298.us-east-1.elb.amazonaws.com -U app_bdp_usage_qa -d cohort -p 7432 << EOF 

SELECT date
 FROM plproxy.execute_select(\$proxy\$
 select date from mw.domain_retention_latest_est_date
 \$proxy\$) t (date DATE) order by date desc limit 3;
 
 SELECT date
 FROM plproxy.execute_select(\$proxy\$
 select date from mw.domain_retention_m
 \$proxy\$) t (date DATE) order by date desc limit 3;
 
 select date from plproxy.execute_select(\$proxy\$select date 
                    from mw.domain_retention_m "
                    "where 
                    retention_days=7  \$proxy\$)t (
                    date DATE) order by date desc limit 3;
EOF




In [0]:
%%sh

# CITUS_AA_CITUS_DB_HOSTS = [('10.2.10.132', 5432)]
# CITUS_AA_CITUS_DB_ACCESS_ID = 'citus_bdp_usage_qa'
# CITUS_AA_CITUS_DB_SECRET_KEY = 'dNzWtSV3pKTx'
# CITUS_AA_CITUS_DB_NAME = 'aa_citus_db'
# CITUS_AA_CITUS_DB_CONNECTION_CAPACITY = 2
# CITUS_AA_CITUS_DB_SEMAPHORE_TIMEOUT = 7201


PGPASSWORD='dNzWtSV3pKTx' psql -h 10.2.10.132  -U citus_bdp_usage_qa -d aa_citus_db -p 5432 << EOF 
set search_path=store;

\d store_est_fact_v2
--select int(5.5) ;


-- select * from store.store_est_fact_v2 where  app_id='835599320' and granularity='daily' and date='2020-03-28' limit 10 ;

SELECT 5.4::integer;
SELECT 5.5::integer;


EOF

In [0]:
%%sh
PGPASSWORD='rn*Wh%osCl2C' psql -h internal-aa-citus-mkt-elb-1511434527.us-east-1.elb.amazonaws.com  -U citus_mkt_qa -d aa_mkt_db -p 7432 << EOF 
set search_path=market;
--select * from market.market_creative_map_dimension_v1 where update_date='2020-07-05'  limit 3;
--select * from market.market_campaign_map_dimension_v1 where update_date='2020-07-05'  limit 3;
--select * from market.market_creative_meta_dimension_v1 where update_date='2020-07-05'  limit 3;

-- select * from market.market_creative_adapp_fact_v1 where date='2020-02-08'  limit 7;
-- select * from market.market_creative_cat_rank_fact_v1  where date='2020-02-08' limit 7;



-- select * from market.market_adapp_pubapp_seen_fact_v1  where first_seen_date='2020-07-07' limit 1 

-- select distinct country_code from market.market_creative_log_fact_v1 where date='2020-07-05' ;
-- select distinct country_code from market.market_creative_adapp_seen_fact_v1 where last_seen_date='2020-07-04' ;

-- select * from market.market_creative_adapp_fact_v2 where date='2020-04-25' AND granularity='weekly'  limit 7;

select * from market.market_adapp_pubapp_fact_v1 where granularity='monthly'  and date='2020-07-31' limit 1 ;
select * from market.market_adapp_cat_rank_fact_v1 where granularity='monthly'  and date='2020-07-31' limit 1 ;

EOF 





In [0]:
%%sh
PGPASSWORD='rn*Wh%osCl2C' psql -h internal-aa-citus-mkt-elb-1511434527.us-east-1.elb.amazonaws.com  -U citus_mkt_qa -d aa_mkt_db -p 7432 << EOF 
set search_path=aso;
--\d
select * from aso.aso_keyword_trafficshare_fact_v1 where date = '2020-08-01' and device_code= 'ios-phone' limit 3;
select * from aso.aso_search_visibility_fact_v1 where date = '2020-08-01' limit 3;
select * from aso.aso_sv_kd_fact_v1  where date = '2020-08-01' limit 3;
select * from aso.aso_sov_search_ads_keyword_fact_v1 where date = '2020-08-01' limit 3;
-- select app_id_list,traffic_share_list from aso.aso_keyword_trafficshare_fact_v1 where date='2020-02-15' and device_code='ios-phone' limit 3
--select * from aso.aso_keyword_trafficshare_fact_v1 where keyword_id='6297851' limit 3

EOF 



In [0]:
%%sh
curl -XGET -u bdp:C38vEJEuraCw  -H 'Content-Type: application/json'   http://internal-internel-review-es-v7-1286999769.us-east-1.elb.amazonaws.com:19200/int-ss-review_v1-*/_search?pretty -d '
{
  "query": {
    "bool": {
      "must": [
        {
          "match": {
            "_id": "6250600663"
          }
        }
        
      ]
    }
  }
}
'

In [0]:
%%sh
curl -XGET -u bdp_read:E9vjhU2qG1bAcM83  -H 'Content-Type: application/json'   http://internal-aa-int-ss-review-elasticsearch-126620990.us-east-1.elb.amazonaws.com:19200/int-reviews-*/gp/_search?pretty -d '
{
  "query": {
    "bool": {
      "must": [
        {
          "match": {
            "date": "2020-01-09"
          }
        },
        {
          "match": {
            "_routing": "20600000009072"
          }
        }
      ]
    }
  }
}
'

In [0]:
%%sh

PGPASSWORD='dNzWtSV3pKTx' psql -h 10.2.10.132 -p 5432 -U citus_bdp_usage_qa -d aa_citus_db << EOF
SET search_path=advancedreview;
-- select * from advancedreview_topic_fact_v3 where date='2020-08-11' limit 2;

select * from advancedreview_term_fact_v3 where date='2020-09-05' and country_code='US' limit 1 ;
select * from advancedreview_term_fact_v3 where date='2020-09-05' and country_code='AU' limit 1 ;

EOF 

In [0]:
%%sh
aws s3 ls s3://b2c-prod-data-pipeline-raw-advancedreview/raw/advancedreview.topic.v1/insert/_default_partition=2020-08/ | sort -n



In [0]:
%%sh

PGPASSWORD='rn*Wh%osCl2C' psql -h internal-aa-citus-mkt-elb-1511434527.us-east-1.elb.amazonaws.com  -U citus_mkt_qa -d aa_mkt_db -p 7432 << EOF 


--SELECT *
--FROM
--    pg_indexes
--WHERE 
--    schemaname = 'aso' AND tablename LIKE 'aso_sov_search_ads_keyword_fact%'
--LIMIT 50
;

select count(*) from aso.aso_sov_search_ads_keyword_fact_v1 where date='2020-05-27' and  device_code='ios-phone' limit 1;


EOF

In [0]:

from elasticsearch import Elasticsearch
import time
import datetime
from pyspark.sql import Row



def prepare_test_data():
    case_list=[]
    start_date=['2019-01-10','2019-12-18','2019-10-19','2019-08-01', '2019-02-01']
    end_date=['2019-12-31']
    date_list=zip(start_date,end_date*5)
    # date_list=zip(start_date,end_date)
    for d in date_list:
        sd,ed=d[0],d[1]
        post_query = {"query":{"constant_score":{"filter":{"bool":{"must":[{"terms":{"category_id":[800000]}},{"terms":{"country_code":["WW"]}},{"range":{"date":{"gte":sd,"lte":ed}}},{"terms":{"granularity":["daily"]}}]}}}},"aggs":{"sdk_id":{"aggs":{"date_count":{"filter":{"range":{"est_sdk_install_base":{"gt":0}}},"aggs":{"date_count":{"cardinality":{"field":"date"}}}},"est_sdk_install_base_total":{"sum":{"field":"est_sdk_install_base"}},"est_sdk_install_base__aggr":{"bucket_script":{"buckets_path":{"count":"date_count>date_count","total":"est_sdk_install_base_total"},"script":"params.total / params.count"}}},"terms":{"field":"sdk_id","size":30000,"collect_mode":"breadth_first"}}},"size":0}
        case_list.append((post_query, sd, ed))
    # print case_list
    return case_list
    
    
# def check_date_size(date_time1, date_time2):
#     if datetime.datetime.strptime(date_time1, "%Y-%m-%d") > datetime.datetime.strptime(date_time2, "%Y-%m-%d"):
#         return date_time2, date_time1
#     else:
#         return date_time1, date_time2
        
        
def get_connection():
    es_connection = Elasticsearch(["http://internal-elb-sdkint-es-test-2058330847.us-east-1.elb.amazonaws.com"],http_auth=("elastic","x0sICMm7ZjOv"),port=19200)
    return es_connection



con_es=get_connection()


def performance_test(params):
    query = params[0]
    start_time=params[1]
    end_time = params[2]

    start = time.time()
    result = con_es.search(index="int_sdk_metric_history_*_ww_daily_2019-*-alias", body=query)
    end= time.time()
    duration = end - start
    result = result["hits"]["total"]["value"]
    elapsed=duration * 1000
    return Row(elapsed=duration * 1000, case='_'.join([start_time, end_time]),result=result, start=start, end=end)

rows=[]
for x in prepare_test_data()*100:
    rows.append(performance_test(x))


df = spark.createDataFrame(rows)
# df.show()
df.createOrReplaceTempView("table1")


# # case_list=[({'query': {'constant_score': {'filter': {'bool': {'must': [{'terms': {'category_id': [800000]}}, {'terms': {'country_code': ['WW']}}, {'range': {'date': {'gte': '2020-01-10', 'lte': '2020-01-16'}}}, {'terms': {'granularity': ['daily']}}]}}}}, 'aggs': {'sdk_id': {'terms': {'field': 'sdk_id', 'collect_mode': 'breadth_first', 'size': 30000}, 'aggs': {'date_count': {'filter': {'range': {'est_sdk_install_base': {'gt': 0}}}, 'aggs': {'date_count': {'cardinality': {'field': 'date'}}}}, 'est_sdk_install_base__aggr': {'bucket_script': {'buckets_path': {'count': 'date_count>date_count', 'total': 'est_sdk_install_base_total'}, 'script': 'params.total / params.count'}}, 'est_sdk_install_base_total': {'sum': {'field': 'est_sdk_install_base'}}}}}, 'size': 0}, '2020-01-10', '2020-01-16')]
# # spark.sparkContext.parallelize(case_list, 1).map(performance_test).toDF().createOrReplaceTempView("table1")



df2 = spark.sql("SELECT count(case) / (max(start) - min(start)) * 60 as rpm, case, avg(result) as result_count, max(elapsed) as max, min(elapsed) as min, avg(elapsed) as avg, percentile(elapsed, 0.95) as 95p from table1 group by case")

# #spark.sql("SELECT * FROM table1").show()

z.show(df2)




In [0]:

import psycopg2
import datetime
spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import aaplproxy
from aadatapipelinecore.core.urn import Urn
from aaplproxy.da.local_sqlrunner import LocalSqlRunner
from aadatapipelinecore.core.utils.module import application_settings
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql import functions as F
def plproxy_row():
    urn = Urn(
        namespace='app-qa.db-check.v1',
        owner='app_qa'
    )
    def build_db_settings(urn, db_conf_name):
        template = "PG_" + db_conf_name.upper() + "_{property}"
        settings = application_settings(urn)
        host, port = getattr(settings, template.format(property='HOSTS'))[0]
        return {
            'HOST': "pg-n-slave-4edc958bda550bbc.elb.us-east-1.amazonaws.com",
            'PORT': "5433",
            'NAME': "aa_android",
            'PASSWORD': "2mHdFW6%#REu",
            'USER': "app_bdp_usage_qa"
            }
    settings = build_db_settings(urn, "AA")
    runner = LocalSqlRunner(settings)
    sql = "SELECT * FROM app LIMIT 2"
    rows, _, columns = runner.select_return_columns(sql)
    print rows

plproxy_row()

